In [1]:
# Import other modules not related to PySpark
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
# This helps auto print out the items without explixitly using 'print'
InteractiveShell.ast_node_interactivity = "all" 
%matplotlib inline

In [2]:
# Import PySpark related modules
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

MAX_MEMORY = '15G'
# Initialize a spark session.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Pyspark guide") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

spark = init_spark()
# Укажите путь к CSV файлу
csv_file_path = "archive/1.csv"

# Прочитайте CSV файл в DataFrame
df = spark.read.csv(csv_file_path, header=True, sep=';', inferSchema=True)

col = ['timestamp','site_id','period_id','actual_consumption','actual_pv', 
       'load_00', 'load_01', 'load_02', 'load_03', 'load_04', 'load_05', 'load_06', 'load_07', 'load_08', 'load_09',
       'pv_00', 'pv_01', 'pv_02', 'pv_03', 'pv_04', 'pv_05', 'pv_06', 'pv_07', 'pv_08', 'pv_09']

df = df[col]
limited_data_pandas = df.limit(10)

limited_data_pandas.toPandas()

,timestamp,site_id,period_id,actual_consumption,actual_pv,load_00,load_01,load_02,load_03,load_04,...,pv_00,pv_01,pv_02,pv_03,pv_04,pv_05,pv_06,pv_07,pv_08,pv_09
0,2014-07-19 18:45:00,1,0,51.625703,22.712489,52.816828,53.501688,54.079161,52.683472,52.590445,...,18.321836,13.912749,10.946568,9.243136,6.962653,5.466993,4.565274,4.082976,3.851306,3.638564
1,2014-07-19 19:30:00,1,0,52.281257,6.618605,51.452796,51.676287,51.329882,51.690879,51.538671,...,6.339899,4.295642,3.016987,2.314616,2.015446,1.952004,1.893801,1.840635,1.791986,1.747604
2,2014-07-19 20:00:00,1,0,50.719565,1.452209,51.313898,52.199835,52.340547,51.844138,52.661063,...,0.936193,0.403129,0.259490,0.338924,0.411971,0.479376,0.541489,0.598855,0.651823,0.700478
3,2014-07-19 20:15:00,1,0,51.901162,0.580877,51.950475,51.624345,50.867434,51.538997,51.331161,...,0.219761,0.091042,0.184182,0.269820,0.348792,0.421529,0.488656,0.550591,0.607483,0.659699
4,2014-07-19 21:00:00,1,0,51.250007,0.000000,52.218820,52.176852,51.745211,51.809854,51.835710,...,0.143507,0.232756,0.314935,0.390735,0.460637,0.524848,0.583788,0.637915,0.687616,0.733272
5,2014-07-19 22:45:00,1,0,51.790326,0.000000,51.857548,51.891930,51.912236,52.005984,51.929889,...,0.170069,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200
6,2014-07-19 23:00:00,1,0,52.460697,0.000000,52.347502,52.285460,52.333257,52.231521,52.220741,...,0.170069,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200
7,2014-07-19 23:15:00,1,0,51.831117,0.000000,52.024522,52.214861,52.192664,52.226268,52.920358,...,0.170069,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200
8,2014-07-19 23:45:00,1,0,52.030533,0.000000,52.202840,52.344978,53.099632,52.468551,52.493247,...,0.170069,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200
9,2014-07-20 00:00:00,1,0,51.754934,0.000000,51.995633,52.805283,52.204891,52.246713,52.270049,...,0.170069,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr
from pyspark.sql.types import IntegerType

# Предположим, что ваш DataFrame называется df
# Замените 'timestamp' на фактическое имя вашей колонки с временем
df = df.withColumn("timestamp", df["timestamp"].cast("timestamp"))

# Добавление новой колонки 'DAY' с условием
df = df.withColumn(
    "DAY",
    when(
        (expr("hour(timestamp) >= 8") & expr("hour(timestamp) < 20")),
        1
    ).otherwise(0)
)

# Если необходимо привести колонку 'DAY' к типу IntegerType
df = df.withColumn("DAY", df["DAY"].cast(IntegerType()))

# Вывод первых нескольких строк DataFrame для проверки результата
limited_data_pandas = df.limit(10)

limited_data_pandas.toPandas()


,timestamp,site_id,period_id,actual_consumption,actual_pv,load_00,load_01,load_02,load_03,load_04,...,pv_01,pv_02,pv_03,pv_04,pv_05,pv_06,pv_07,pv_08,pv_09,DAY
0,2014-07-19 18:45:00,1,0,51.625703,22.712489,52.816828,53.501688,54.079161,52.683472,52.590445,...,13.912749,10.946568,9.243136,6.962653,5.466993,4.565274,4.082976,3.851306,3.638564,1
1,2014-07-19 19:30:00,1,0,52.281257,6.618605,51.452796,51.676287,51.329882,51.690879,51.538671,...,4.295642,3.016987,2.314616,2.015446,1.952004,1.893801,1.840635,1.791986,1.747604,1
2,2014-07-19 20:00:00,1,0,50.719565,1.452209,51.313898,52.199835,52.340547,51.844138,52.661063,...,0.403129,0.259490,0.338924,0.411971,0.479376,0.541489,0.598855,0.651823,0.700478,0
3,2014-07-19 20:15:00,1,0,51.901162,0.580877,51.950475,51.624345,50.867434,51.538997,51.331161,...,0.091042,0.184182,0.269820,0.348792,0.421529,0.488656,0.550591,0.607483,0.659699,0
4,2014-07-19 21:00:00,1,0,51.250007,0.000000,52.218820,52.176852,51.745211,51.809854,51.835710,...,0.232756,0.314935,0.390735,0.460637,0.524848,0.583788,0.637915,0.687616,0.733272,0
5,2014-07-19 22:45:00,1,0,51.790326,0.000000,51.857548,51.891930,51.912236,52.005984,51.929889,...,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200,0
6,2014-07-19 23:00:00,1,0,52.460697,0.000000,52.347502,52.285460,52.333257,52.231521,52.220741,...,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200,0
7,2014-07-19 23:15:00,1,0,51.831117,0.000000,52.024522,52.214861,52.192664,52.226268,52.920358,...,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200,0
8,2014-07-19 23:45:00,1,0,52.030533,0.000000,52.202840,52.344978,53.099632,52.468551,52.493247,...,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200,0
9,2014-07-20 00:00:00,1,0,51.754934,0.000000,51.995633,52.805283,52.204891,52.246713,52.270049,...,0.257836,0.338463,0.412528,0.480568,0.543071,0.600489,0.653234,0.701688,0.746200,0


<h2 style="text-align:center;font-size:200%;;">GradientBoostingMachine</h2>

In [11]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Выберем только нужные колонки
selected_columns = ['actual_consumption', 'load_00', 'load_01', 'load_02']
data_subset = df.select(selected_columns)

# Создадим столбец features, который объединяет все признаки в один вектор
assembler = VectorAssembler(inputCols=selected_columns[1:], outputCol="features")
data_subset = assembler.transform(data_subset)

# Создадим столбец label, который содержит значения actual_consumption
data_subset = data_subset.withColumnRenamed("actual_consumption", "label")

# Разделим данные на обучающую и тестовую выборки
(training_data, test_data) = data_subset.randomSplit([0.8, 0.2], seed=42)

# Инициализация модели GBTRegressor
gbt = GBTRegressor(featuresCol="features", labelCol="label", seed=42)

# Создадим сетку параметров для подбора
param_grid = (ParamGridBuilder()
              .addGrid(gbt.maxDepth, [5, 5])
              .addGrid(gbt.maxIter, [10, 10])
              .build())

# Инициализация оценщика (evaluator)
evaluator_mse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
# evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
# evaluator_mae = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
# evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

# Инициализация кросс-валидации
cross_val = CrossValidator(estimator=gbt, estimatorParamMaps=param_grid, evaluator=evaluator_mse, numFolds=3)

# Обучение модели на обучающей выборке с кросс-валидацией
cv_model = cross_val.fit(training_data)

# Прогнозирование на тестовой выборке
predictions = cv_model.transform(test_data)

# Вывод предсказаний и выбранных признаков
predictions.select("label", "prediction", *selected_columns[1:]).show()

# Оценка качества модели - MSE
mse_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
mse = mse_evaluator.evaluate(predictions)
print(f"Mean Squared Error: {mse}")

mae_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(predictions)
print(f"Mean Absolute Error: {mae}")

rmse_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = rmse_evaluator.evaluate(predictions)
print(f"Root Mean Squared Error: {rmse}")

r2_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
r2 = r2_evaluator.evaluate(predictions)
print(f"R^2: {r2}")


# Вывод лучших параметров
best_params = cv_model.bestModel.extractParamMap()
print("Best Parameters:")
for param, value in best_params.items():
    print(f"{param.name}: {value}")

+------------------+------------------+------------------+------------------+------------------+
|             label|        prediction|           load_00|           load_01|           load_02|
+------------------+------------------+------------------+------------------+------------------+
| 46.37877351486006|51.924016330812094| 50.28615615753178|53.103660442223465|54.021746474461594|
| 47.66812587756288| 51.88519638467783|  51.7386783264035|  52.4157422429756| 54.91779594308641|
|47.964859680213365|52.656905564661635| 50.43052885212892|55.400671804201885| 55.13205430837825|
| 48.33161786135996| 52.68538671020847|52.466679351006526| 52.19508805391409| 52.98176274841146|
|48.698975908684446| 52.99973391859037| 49.60818758737901| 50.51739926607357|51.426610944768136|
| 48.91568756320776| 53.62335808029519| 52.34697007958418| 52.86800756502827| 50.49467548775932|
| 49.14219703196954| 53.08712750323669| 52.92301554354659| 54.33619353171711|  54.2822371013329|
| 49.44304991570806| 52.228496

- `cacheNodeIds: False`: Не кешировать идентификаторы узлов.
- `checkpointInterval: 10`: Интервал между контрольными точками.
- `featureSubsetStrategy: all`: Все признаки используются при обучении каждого дерева.
- `featuresCol: features`: Имя столбца, содержащего признаки.
- `impurity: variance`: Критерий оценки качества разделения дерева (дисперсия).
- `labelCol: label`: Имя столбца, содержащего целевую переменную.
- `maxBins: 32`: Максимальное количество корзин для категориальных признаков.
- `maxDepth: 5`: Максимальная глубина каждого дерева.
- `maxIter: 20`: Максимальное количество итераций (деревьев) в градиентном бустинге.
- `minInfoGain: 0.0`: Минимальное значение информационного выигрыша для узла дерева.
- `minInstancesPerNode: 1`: Минимальное количество экземпляров для создания узла дерева.
- `minWeightFractionPerNode: 0.0`: Минимальная доля веса суммируемого экземпляра для создания узла дерева.
- `predictionCol: prediction`: Имя столбца, содержащего предсказания модели.
- `seed: 42`: Зерно для случайной инициализации.
- `stepSize: 0.1`: Размер шага для оптимизации.
- `subsamplingRate: 1.0`: Доля данных, используемых для обучения каждого дерева.
- `validationTol: 0.01`: Требуемая относительная терпимость для оценки производительности на валидционном наборе.
ационном наборе.

Гиперпараметры:
- `maxDepth`: Максимальная глубина каждого дерева.
- `maxIter`: Максимальное количество итераций (деревьев) в градиентном бустинге.
- `minInfoGain`: Минимальное значение информационного выигрыша для узла дерева.
- `minInstancesPerNode`: Минимальное количество экземпляров для создания узла дерева.
- `minWeightFractionPerNode`: Минимальная доля веса суммируемого экземпляра для создания узла дерева.
- `stepSize`: Размер шага для оптимизации.


<h2 style="text-align:center;font-size:200%;;">LogisticRegression</h2>

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Инициализация SparkSession
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# Здесь предполагается, что df - ваш исходный DataFrame

# Выберем только нужные колонки
selected_columns = ['DAY', 'actual_consumption', 'load_00', 'load_01', 'load_02']
data_subset = df.select(selected_columns)

# Переименование столбца DAY в label
data_subset = data_subset.withColumnRenamed("DAY", "label")

# Создание стадий конвейера
vector_assembler = VectorAssembler(inputCols=selected_columns[1:], outputCol="features")
scaler = MinMaxScaler(inputCol='features', outputCol='featureNorm')
lr = LogisticRegression(featuresCol="featureNorm", labelCol="label")

# Создание конвейера
pipeline = Pipeline(stages=[vector_assembler, scaler, lr])

# Разделение данных на обучающую и тестовую выборки
(train_data, test_data) = data_subset.randomSplit([0.8, 0.2], seed=42)

# Оценка качества модели
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

# Определение сетки параметров для кросс-валидации
param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Создание объекта CrossValidator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)  # Указание количества складок для кросс-валидации

# Обучение модели на обучающей выборке с использованием кросс-валидации
cv_model = crossval.fit(train_data)

# Прогнозирование на тестовой выборке
predictions_cv = cv_model.transform(test_data)

# Оценка качества модели
area_under_roc_cv = evaluator.evaluate(predictions_cv)
print(f"Area under ROC curve (cross-validated): {area_under_roc_cv}")

# Метрики классификации
true_positives = predictions_cv.filter("prediction = 1.0 AND label = 1").count()
true_negatives = predictions_cv.filter("prediction = 0.0 AND label = 0").count()
false_positives = predictions_cv.filter("prediction = 1.0 AND label = 0").count()
false_negatives = predictions_cv.filter("prediction = 0.0 AND label = 1").count()

# Accuracy (точность)
accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
print(f"Accuracy: {accuracy}")

# Precision (точность)
precision = true_positives / (true_positives + false_positives)
print(f"Precision: {precision}")

# Recall (полнота)
recall = true_positives / (true_positives + false_negatives)
print(f"Recall: {recall}")

# F1 Score (F-мера)
f1_score = 2 * (precision * recall) / (precision + recall)
print(f"F1 Score: {f1_score}")

# ROC-AUC (площадь под ROC-кривой) - для бинарной классификации
# В данном случае, с использованием PySpark, уже было вычислено area_under_roc_cv

# Вывод матрицы ошибок (Confusion Matrix)
print("Confusion Matrix:")
print(f"True Positives: {true_positives}")
print(f"True Negatives: {true_negatives}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")


Area under ROC curve (cross-validated): 0.8120909307878146
Accuracy: 0.8034531436638072
Precision: 0.9090373859287607
Recall: 0.6729134887775114
F1 Score: 0.7733533683946907
Confusion Matrix:
True Positives: 3088
True Negatives: 4311
False Positives: 309
False Negatives: 1501


<h2 style="text-align:center;font-size:200%;;">Выводы</h2>

В рамках лабораторной работы было проведено машинное обучение на больших данных с использованием PySpark для датасета исторических данных по фотоэлектричеству и нагрузке. В ходе работы были выполнены следующие задачи:

- Ознакомление с базовыми алгоритмами машинного обучения;
- Ознакомление с реализацией машинного обучения в библиотеке Spark ML;
- Получение навыков разработки программного обеспечения для анализа данных с использованием PySpark;
- Решение задачи регрессии с использованием GradientBoostingMachine;
- Решение задачи бинарной классификации с использованием LogisticRegression.
